In [4]:
import instructor
import vertexai  # type: ignore
from vertexai.generative_models import GenerativeModel  # type: ignore
from pydantic import BaseModel
from dotenv import load_dotenv

load_dotenv()

# Initialize Vertex AI client
vertexai.init()

# Define your structured response schema
class User(BaseModel):
    name: str
    age: int

# Wrap the Vertex GenerativeModel in Instructor
client = instructor.from_vertexai(
    client=GenerativeModel("gemini-2.5-flash-preview-04-17"),
    mode=instructor.Mode.VERTEXAI_TOOLS,
)


In [5]:
dir(client)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'chat',
 'clear',
 'client',
 'completions',
 'create',
 'create_fn',
 'create_iterable',
 'create_partial',
 'create_with_completion',
 'default_model',
 'handle_kwargs',
 'hooks',
 'kwargs',
 'messages',
 'mode',
 'off',
 'on',
 'provider']

In [6]:
# Create a structured response synchronously
resp = client.create(
    messages=[{"role": "user", "content": "Extract Jason is 25 years old."}],
    response_model=User,
)
resp


User(name='Jason', age=25)

In [8]:
client.create

<bound method Instructor.create of <instructor.client.Instructor object at 0x7f3fc2f7f190>>

In [21]:
from pydantic import BaseModel, Field

class AddInput(BaseModel):
    """add two numbers"""
    x: int = Field(..., description="First addend")
    y: int = Field(..., description="Second addend")

class AddResult(BaseModel):
    result: int = Field(..., description="Sum of x and y")


In [24]:
from instructor import openai_schema

tool_schema = openai_schema(AddInput)


In [25]:
tool_schema.openai_schema

{'name': 'AddInput',
 'description': 'add two numbers',
 'parameters': {'properties': {'x': {'description': 'First addend',
    'title': 'X',
    'type': 'integer'},
   'y': {'description': 'Second addend', 'title': 'Y', 'type': 'integer'}},
  'required': ['x', 'y'],
  'type': 'object'}}

In [26]:
input: AddInput = client.create(
    messages=[{"role": "user", "content": "Please add 7 and 13"}],
    tools=[{"type": "function", "function": tool_schema}],
    response_model=AddInput,
)

input

AddInput(x=7, y=13)

In [27]:
def add(x: int, y: int) -> int:
    """Add two numbers"""
    return x + y

In [30]:
add(**input.model_dump())

20